# HW8



Here is a Linked list class (some of which was done few labs/classes back) . We'll use a linked list as a repository for bindings. We will consider an implementation where newer bindings for existing variables occur closer to the head in the linked list. Notice that we added a `repOK` which is an identity function. Take care to see where it is used.

In [63]:
import reprlib, numbers


class LL:
    """
    >>> A = LL()  
    >>> A[0]
    Traceback (most recent call last):
        ...
    IndexError: trying to index an empty LL
    >>> A.insert_front(1)
    >>> A[0]
    1
    >>> A.insert_back(2)
    >>> A[1]
    2
    >>> A
    LL([1,...])
    >>> myll = LL.from_components([1,2])
    >>> myll[1]
    1
    >>> len(myll)
    2
    >>> myll[2]
    Traceback (most recent call last):
        ...
    IndexError: LL index out of range
    >>> myll[0:1]
    Traceback (most recent call last):
        ...
    TypeError: LL indices must be integers
    """
    @classmethod
    def from_components(cls, components):
        inst = cls(components[0])
        for c in components[1:]:
            inst.insert_front(c)
        return inst
        
    def repOK(self, element):
        return element
    
    def __init__(self, head=None):
        if head is None:
            self._headNode = None
        else:
            head = self.repOK(head)
            self._headNode = [head, None]
            
    def insert_front(self, element):
        element = self.repOK(element)
        new_node = [element, None]
        new_node[1] = self._headNode
        self._headNode = new_node
        
    def insert_back(self, element):
        element = self.repOK(element)
        new_node = [element, None]
        curr_ptr = self._headNode
        while curr_ptr[1] is not None:
            curr_ptr = curr_ptr[1]
        curr_ptr[1]= new_node
        
    def __repr__(self):
        class_name = type(self).__name__
        if len(self)==0:
            components=""
        else:
            components = reprlib.repr(self[0])
        return '{}([{},...])'.format(class_name,components)


    def __len__(self):
        curr_ptr = self._headNode
        count = 0
        if curr_ptr==None:
            return 0
        while 1:
            count = count + 1
            if curr_ptr[1] is None:
                break
            curr_ptr = curr_ptr[1]
        return count    
    
    def __getitem__(self, index):
        class_name = type(self).__name__
        if isinstance(index, numbers.Integral): 
            curr_ptr = self._headNode
            if curr_ptr==None:
                msg = 'trying to index an empty {class_name}' 
                raise IndexError(msg.format(class_name=class_name))
            next_ptr = self._headNode[1]
            count = 0
            while 1:
                if index == count:
                    return curr_ptr[0]
                if curr_ptr[1] is None:
                    msg = '{class_name} index out of range' 
                    raise IndexError(msg.format(class_name=class_name))       
                count += 1
                curr_ptr = curr_ptr[1]
        else:
            msg = '{class_name} indices must be integers' 
            raise TypeError(msg.format(class_name=class_name))
         
    def index(self, element):
        class_name = type(self).__name__
        curr_ptr = self._headNode
        count = 0
        if curr_ptr==None:
            msg = 'trying to get index from empty {class_name}' 
            raise IndexError(msg.format(class_name=class_name))
        while 1:
            if curr_ptr[0] == element:
                return count
            if curr_ptr[1] is None:
                msg = '{element} is not in {class_name}' 
                raise ValueError(msg.format(element=element, class_name=class_name))
            count += 1
            curr_ptr = curr_ptr[1]
            
    def remove(self, element):
        class_name = type(self).__name__
        curr_ptr = self._headNode
        prev_ptr = None
        if curr_ptr==None:
            msg = 'remove from empty {class_name}' 
            raise IndexError(msg.format(class_name=class_name))
        while 1:
            if curr_ptr[0] == element:
                if prev_ptr is None:
                    self._headNode = curr_ptr[1]
                else:
                    prev_ptr[1] = curr_ptr[1]
                return None
            if curr_ptr[1] is None:
                msg = '{element} is not in {class_name}' 
                raise ValueError(msg.format(element=element, class_name=class_name))
            prev_ptr=curr_ptr
            curr_ptr = curr_ptr[1]
        
    def remove_front(self):
        class_name = type(self).__name__
        curr_ptr = self._headNode
        if curr_ptr==None:
            msg = 'remove from empty {class_name}' 
            raise IndexError(msg.format(class_name=class_name))
        self._headNode = curr_ptr[1]
        return curr_ptr[0]
    

In [64]:
import abc
class Environment(abc.ABC):
    """
    This is the interface for an Environment. The client for 
    this interface is a language intepreter. 
    """
    @classmethod
    @abc.abstractmethod
    def empty(cls):
        return cls()
    
    @abc.abstractmethod
    def extend(self, variable, value):
        """
        extend an existing environment by binding variable to value.
        The values must be an acceptable value in the language. If the
        same variable is used twice the newer value must be bound.
        """
    
    @abc.abstractmethod
    def extend_many(self, envdict):
        """
        extend the current environment by values in the dictionary
        envdict. If the dictionary contains variables already in the
        environment, the newer values from the dictionary are bound
        """
    @abc.abstractmethod
    def lookup(self, variable):
        """
        return the unique binding of the variable and the environment it was bound
        in as a tuple. If it is not found raise a NameError as below
        """
        raise NameError("{} not found in Environment".format(variable))


### Q1

Lets inherit from class `LL` to create a TupleLL where the values in the linked list are tuples of the type `(key, value)`. After inheriting, add an additional method `first_match` which returns a tuple `(index_of_nearest_element, nearest element)` to the head where the first member of the tuple matches `key`. Also add another method `all_matches` which returns a list of tuples of allmatches, sorted by distance to head. Currently, the structure of the other methods in the linked list do not need to know about this 2-element tuple restriction on the elements. 

In [65]:
class TupleLL(LL):
    """
    A linked list whose elements must be tuples.
    
    RepInv: any insertion should leave a list with only tuples as members.
    """
    #could be implemented using a check on the whole list
    def repOK(self, element):
        assert isinstance(element, tuple), "element needs to be a tuple"
        return element
    
    #your code here
    def first_match(self, key):
        class_name = type(self).__name__
        if isinstance(key, str): 
            curr_ptr = self._headNode
            if curr_ptr==None:
                return None
            
            next_ptr = self._headNode[1]
            count = 0
            while 1:
                if key == curr_ptr[0][0]:
                    return (count, curr_ptr[0])
                if curr_ptr[1] is None:
                    return None
                
                count += 1
                curr_ptr = curr_ptr[1]
        else:
            msg = '{class_name} key must be string' 
            raise TypeError(msg.format(class_name=class_name))
        
        
    def all_matches(self, key):
        class_name = type(self).__name__
        
        if isinstance(key, str): 
            results = []
            curr_ptr = self._headNode
            if curr_ptr==None:
                return results

            next_ptr = self._headNode[1]
            count = 0
            while 1:
                if key == curr_ptr[0][0]:
                    results.append((count, curr_ptr[0]))
                if curr_ptr[1] is None:
                    return results

                count += 1
                curr_ptr = curr_ptr[1]
        else:
            msg = '{class_name} key must be string' 
            raise TypeError(msg.format(class_name=class_name))

In [25]:
l = TupleLL()
l.insert_front(('a', 1))
l.insert_front(('b',2))
l.insert_front(('c',3))
l.insert_front(('b',6))

In [33]:
# l.first_match('c')
a = l.first_match('a')
a[1]
# l.first_match('a')
# l.insert_front(3)

('a', 1)

(1, 2)

### Q2.

Implement an implementation class `Env2` that impements the environment interface using the `TupleLL`. Let us ask what the AbsFun and RepInv are for this implementation. Write an AbsFun for it, noting that in this concrete representation, there can be multiple key-value pairs in the environment for the same key, and that there is a way to disambiguate the correct one. 

(In general, the Absfun should limit the keys and values allowed for our bindings (as according to our language) as well. But we wont do that here.)




In [68]:
#your code here
class Env2(Environment):
        
    def __init__(self):
        self.env = TupleLL()

    def empty(cls):
        return cls()
    
    def extend(self, variable, value):
        """
        extend an existing environment by binding variable to value.
        The values must be an acceptable value in the language. If the
        same variable is used twice the newer value must be bound.
        """
        match = self.env.first_match(variable)
        if match:
            matchTuple = match[1]
            # Bind new value to variable
            self.env.remove((matchTuple[0], matchTuple[1]))
            self.env.insert_front((variable, value))
        else:
            self.env.insert_front((variable, value))
    
    def extend_many(self, envdict):
        """
        extend the current environment by values in the dictionary
        envdict. If the dictionary contains variables already in the
        environment, the newer values from the dictionary are bound
        """
        for k, v in envdict.items():
            self.extend(k, v)

    def lookup(self, variable):
        """
        return the unique binding of the variable and the environment it was bound
        in as a tuple. If it is not found raise a NameError as below
        """
        match = self.env.first_match(variable)
        if match:
            # match = (index, (var, val))
            return match[1]
        
        raise NameError("{} not found in Environment".format(variable))


In [71]:
# Simple test case
e = Env2()
e.extend('a', 1)
e.extend('b', 2)
e.extend('a', 4)
e.extend_many({'a':5, 'c':6, 'd':9})
e.lookup('b')

('b', 2)

### Q3. `repinv`

Now let us write and test a RepInv for this implementation. Note that just like in the case with the list-with-duplicates implementation of a set, its hard to come up with any uniqueness-of-binding RepInv. But we can ask the following: a newer binding overrides an old one. Write a Repinv which implements this and include a repOK methodwhich makes sure that any methods mutating the environment respect this RepInv and any observers return the correct binding. 

Hint: the `repOK` will need a signature `def repOK(self, key, value)` and also be used in the `lookup` method. It will use all_matches, and not be returning anything.

In [121]:
#your code here
#your code here
class Env2(Environment):
        
    def __init__(self):
        self.env = TupleLL()

    @classmethod
    def empty(cls):
        return cls()
    
    def extend(self, variable, value):
        """
        extend an existing environment by binding variable to value.
        The values must be an acceptable value in the language. If the
        same variable is used twice the newer value must be bound.
        """
        match = self.env.first_match(variable)
        if match:
            matchTuple = match[1]
            # Bind new value to variable
            self.env.remove((matchTuple[0], matchTuple[1]))
            self.env.insert_front((variable, value))
        else:
            self.env.insert_front((variable, value))
            
        # The env should only have variable, value pair
        self.repOK(variable, value)
    
    def extend_many(self, envdict):
        """
        extend the current environment by values in the dictionary
        envdict. If the dictionary contains variables already in the
        environment, the newer values from the dictionary are bound
        """
        for k, v in envdict.items():
            self.extend(k, v)

    def lookup(self, variable):
        """
        return the unique binding of the variable and the environment it was bound
        in as a tuple. If it is not found raise a NameError as below
        """
        match = self.env.first_match(variable)
        if match:
            # match = (index, (var, val))
            return (match[1][1], self.env)
        
        raise NameError("{} not found in Environment".format(variable))

    def repOK(self, key, value):
        matches = [t[1] for t in self.env.all_matches(key)] # [(k1,v1), (k2,v2) ...]
        if key != 'nan':
            assert(len(matches) == 1)
            assert (matches[0][0] == key and matches[0][1] == value)

In [107]:
print(float('nan') == float('nan'))

False


### Q4. Running the program

Just make sure that the program actually runs :-). Note some changes here: we are using `yield`s instead of `return`s at multiple places.

Lets use this class in a function which creates a global environment for our calculator referenced as `globenv`. we first register this implementation for the `Environment` interface

In [122]:
Environment.register(Env2)

__main__.Env2

In [123]:
import math
def global_env(envclass):
    "An environment with some Scheme standard procedures."
    import math, operator as op
    print("envclass: ", envclass)
    env = envclass.empty()
    env.extend_many(vars(math))
    env.extend_many({
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv, 
        'abs':     abs,
        'max':     max,
        'min':     min,
        'round':   round,
        '>':op.gt, '<':op.lt, '>=':op.ge, '<=':op.le, '==':op.eq,
        'not':     op.not_
    })
    return env

In [124]:
class Function():
    def __init__(self, params, parsedbody, env):
        self.params = params
        self.code = parsedbody
        self.env = env
        self.envclass = env.__class__
        
    def __call__(self, *args):
        funcenv = self.envclass(outerenv = self.env)
        funcenv.extend_many(zip(self.params, args))
        return eval_ptree(self.code, funcenv)

In [125]:
globenv = global_env(Env2)

envclass:  <class '__main__.Env2'>


In [126]:
def typer(token):
    try:
        t = int(token)
        return t
    except ValueError:
        try:
            t = float(token)
            return t
        except ValueError:
            return str(token)
        
def lex(loc):
    tokenlist =  loc.replace('(', ' ( ').replace(')', ' ) ').split()
    return [typer(t) for t in tokenlist]

def syn(tokens):
    if len(tokens) == 0:
        return []
    token = tokens.pop(0)
    if token == '(':
        L = []
        while tokens[0] != ')':
            L.append(syn(tokens))
        tokens.pop(0) # pop off ')'
        return L
    else:
        if token==')':
            assert 1, "should not have got here"
        return token
    
def parse(loc):
    return syn(lex(loc))

In [127]:
def eval_ptree(x, env):
    fmap={'#t':True, '#f':False, 'nil':None}
    Symbol = str
    if x in ('#t', '#f', 'nil'):
        return fmap[x]        
    elif isinstance(x, Symbol):
        # variable lookup
        return env.lookup(x)[0]
    elif not isinstance(x, list):  # constant
        return x
    elif len(x)==0: #noop
        return None
    elif x[0]=='if':
        (_, predicate, truexpr, falseexpr) = x
        if eval_ptree(predicate, env):
            expression = truexpr
        else:
            expression = falseexpr
        return eval_ptree(expression, env)
    elif x[0] == 'def':         # variable definition
        (_, var, expression) = x
        #postorder traversal by nested eval is needed below
        # your code here
        env.extend(var, eval_ptree(expression, env))
    elif x[0] == 'store':           # (set! var exp)
        (_, var, exp) = x
        env.lookup(var)[1].extend(var, eval_ptree(exp, env))
    elif x[0] == 'func':
        (_, parameters, parsedbody) = x
        return Function(parameters, parsedbody, env)
    else:                          # operator
        op = eval_ptree(x[0], env)
        #postorder traversal to get subexpressione before running the op
        args = [eval_ptree(arg, env) for arg in x[1:]]
        return op(*args)

In [128]:
class Program():
    
    def __init__(self, program, env):
        self.program = [e.strip() for e in program.split('\n')]
        self.env = env
        
    def __iter__(self):
        for line in self.program:
            yield line
    
    def parse(self):
        for l in iter(self):
            yield parse(l)
            
    def run(self):
        for l in iter(self):
            yield eval_ptree(parse(l), self.env)

In [129]:
program = """
(def radius 5)
(* pi (* radius radius))
"""

In [130]:
p=Program(program, globenv)
list(iter(p))

['', '(def radius 5)', '(* pi (* radius radius))', '']

In [131]:
for s in p.parse():
    print(s)

[]
['def', 'radius', 5]
['*', 'pi', ['*', 'radius', 'radius']]
[]


In [132]:
for result in p.run():
    print(result)

None
None
78.53981633974483
None
